In [1]:
import pooch
import xarray as xr

In [7]:
# get higher memory local dask workers (by not using all cores on the server)
from distributed import Client, LocalCluster

# cluster = LocalCluster(n_workers=1, threads_per_worker=1)
# client = Client(cluster)
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/jbusecke/leap_data/proxy/8787/status,
Dashboard: /user/jbusecke/leap_data/proxy/8787/status,Workers: 4
Total threads: 16,Total memory: 58.87 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34047,Workers: 4
Dashboard: /user/jbusecke/leap_data/proxy/8787/status,Total threads: 16
Started: Just now,Total memory: 58.87 GiB
Comm: tcp://127.0.0.1:45935,Total threads: 4
Dashboard: /user/jbusecke/leap_data/proxy/40813/status,Memory: 14.72 GiB
Nanny: tcp://127.0.0.1:38575,


In [52]:
fid_dict = {
    'CanESM2': {
        'r1r9': [38264412, 'd64928f54d0cc7d46bf6efd24f31b8dc'],
        'r3r1': [38264427, '10b445c7ccef35218e072295b8c3001c'],
        'r4r5': [38264463, '8ff3eb0fb642b10423f72f91ef2c8b19'],
        'r1r10': [38264415, '492d4ff4da8331697b8eb19100b38d07'],
        'r5r10': [38264496, 'c376360663fa13e6e46d60f85c2c1d4e'],
        'r2r1': [38264664, 'd975cc6d3686cce1c8460dea7ae1d64e'],
        'r3r2': [38264430, 'b5c5562f2e7b40bab018e3bc787a6e49'],
        'r3r9': [38264448, 'de4c12917f8014ac7f36a1592263f957'],
        'r4r6': [38264469, 'db3a2d89eca2d6fbec74815523771b00'],
        'r5r2': [38264481, 'f78400948cf1ac1f20974fe198dab5b1'],
        'r1r6': [38264403, 'f2ddb295efb1080329fe5ba97a1d5426'],
        'r2r2': [38264667, '008f99c2cb2f8b2c5b46500dd7192be7'],
        'r3r4': [38264433, '0c746aab0b146ea5d76502b184899186'],
        'r4r1': [38264454, '99bc46e38e0ce1537e00eb18ab478db2'],
        'r4r7': [38264472, '913b21bf6a7765e68a4ba33d2b32d161'],
        'r5r4': [38264487, '7cbe9df0edbeb5c020139183df4e38e4'],
        'r1r7': [38264406, '0086e6bb93ef9453c0089c4e061a22d2'],
        'r3r6': [38264439, 'cd39e4ca4179cabaf3f05ede5c972517'],
        'r4r8': [38264475, '960fc43b6b38d280315b328a4888f727'],
        'r5r5': [38264490, '3118a2aa198aedbeeafbe44f76491a3a'],
        'r2r8': [38264670, '9f4d649cff9a9b20f502c2679c8c424d'],
        'r3r7': [38264445, '8f31db18968ca1a317d71821e3764e66'],
        'r4r3': [38264457, 'b091609e9356124e9c8a025e21c96311'],
        'r5r1': [38264478, '3bf89c42989bb11088414f784632a8e2'],
        'r5r9': [38264493, '15d621758b76d476850a5b628540f770'],
    },
    "CESM": {
        # "001": [38251560, '4f6729125d9fca4a4b7a33f95a1cb883'], # this one causes an exception when loading later.
        "002": [38251569, '642df7ca94599347271a8622ed2dc17b'],
        "009": [38251572, '08e2f94c0ec303a542d4818a201ac1b0'],
        "010": [38251578, '04a2c3ba74b3247291762cc89344536f'],
        "011": [38251584, '6de42f2ebd674046f85378b922e3c4a6'],
        "012": [38251593, '1305ef0de482138e886287e102cad3ac'],
        "013": [38251596, 'fe2543e418c85beb88fd4318efc3b6e0'],
        "014": [38251599, 'c25103eacef476b47055b494ba1557bb'],
        "015": [38251602, '7da10e2b95160248102f0fe8917b1ce7'],
        "016": [38251608, 'f527105fa4b9721eadc998e2704a09df'],
        "017": [38251614, '56e378696faccb6b64a416b234b1a7b7'],
        "018": [38251620, 'e275a2b5dead260258d22163eaeea092'],
        "020": [38251626, 'b3c789587f6ed4c047290b7c231433f7'],
        "021": [38251629, 'd6d6b5c0024f12dbb24e799a6ee920a8'],
        "023": [38251638, '898c22ec777d04d9b7717eede8fb2e27'],
        "024": [38251653, '5ba27faa874effb94f681141570d7bc8'],
        "025": [38251662, 'a5da1b1e84c7bbd310364fc0af81d936'],
        "030": [38251665, 'ca6f170e51eb2105a902934836c09b2b'],
        "031": [38251671, '4161533673fd9fde51fa23c2426b37e6'],
        "034": [38251677, '158255b157bbf1ceed5a42082d12c4be'],
        "035": [38251692, '895a4b459d176bc914878e6724ab9ca8'],
        "101": [38251695, 'de2805a1dde9e51a7affaca5e10bf8d7'],
        "102": [38251698, '772084dca422173ce3c9aad6b9b74667'],
        "103": [38251704, '8b5ce5a6cb66be94bf73a959fdacf3e5'],
        "104": [38251719, '40e8f6cff00a96c293422d2b5eda9ebc'],
    },
    "GFDL": {
        "01": [38265117, '9412f2560722840f6ef86b80f952076e'],
        "02": [38265123, '6b926dd64a9265551b6436f2ef61c95a'],
        "03": [38265129, 'd492c129dd34a802f84993716a6e633f'],
        "04": [38265141, '4ab116505e988340a6915e5cdecf0545'],
        "05": [38265144, 'd1eca5b93d8cc471f0dfea93389ce0b7'],
        "06": [38265150, '7bca5d62c28b5348682db9561c19118a'],
        "08": [38265153, '2de5c7ed5365bf49d0eb7767bb0ca41e'],
        "09": [38265156, 'fea7d0c6bb6ce1ed0af1045f896fc0bb'],
        "10": [38265159, 'def28866138eb608cdebc01b4688f794'],
        "11": [38265165, '793fdaf001945bb83201cb0793f7ded1'],
        "12": [38265180, 'd13c1c6b83b4e9fef254144267d8f482'],
        "13": [38265183, '7d461127b0d928eab28fea14d4c57c1d'],
        "14": [38265186, 'b3678816dd454db558e9b7356d498735'],
        "16": [38265192, '3319bcbedb478ae94bd5d98f92244e81'],
        "17": [38265198, '4ec75af35057883012fd9ceb62f6f094'],
        "18": [38265201, '0369780ec19d80f2ea238ee4eaf1c1be'],
        "19": [38265204, '06730b6c3c83b701778a2a305d74bb64'],
        "20": [38265210, '5ff73a6396d6726653bcb7a5e7a978df'],
        "22": [38265213, 'cccb0770da34b17465fb86ce1f78fa92'],
        "23": [38265216, 'de006a5c8b10eacb0cf0f078aa7df9f4'],
        "26": [38265222, '493f130479bd841d70973fbce6ae66b8'],
        "27": [38265228, '79db5b7c1fe94e65b322d7baeb8c063c'],
        "28": [38265234, '570ac16e0f585411e8b5ac893ff0df57'],
        "29": [38265237, '1a85c54f80e83bf639dcb44ebb210489'],
        "30": [38265240, 'd6c88a3ab04e13da4bf8095aadedcbe4'],
    },
}

In [53]:
# would be nice to do this in async, especially with more members. Might also be overkill tho...

In [54]:
def fix_cal(ds):
    ds.time.attrs = {
        'standard_name': 'time',
        'long_name': 'time',
        'bounds': 'time_bnds',
        'axis': 'T',
    }
    return ds

In [55]:
import pooch
from tqdm.auto import tqdm

pooch_testbed = {}
file_dict = {}
dataset_dict = {}

for model in tqdm(fid_dict.keys(), desc='Models'):
    pooch_testbed[model] = pooch.create(
        # Use the default cache folder for the operating system
        path=pooch.os_cache(f"LETestbed_{model}"),
        base_url="https://figshare.com/ndownloader/files/",
        # The registry specifies the files that can be fetched
        registry={str(id): f"md5:{hash}" for id, hash in fid_dict[model].values()},
    )
    file_dict[model] = {}
    for f in tqdm(pooch_testbed[model].registry_files, desc='Download Files'):
        file_dict[model][f] = pooch_testbed[model].fetch(
            f, processor=pooch.Unzip(), progressbar=False
        )

    dataset_dict[model] = {}
    for fid, files in tqdm(file_dict[model].items(), desc='Reading Files'):
        # get the member_id from filename
        (member,) = set([path.split('.unzip/')[-1].split('/')[0] for path in files])

        # open all files into an xarray dataset
        ds = xr.open_mfdataset(
            files,
            # decode_times=False,
            chunks={'time': 200},
            compat='override',
            coords='override',
            join='override',
            use_cftime=True,
        )

        # check that all variables we want are there
        required_variables = [
            'Chl_socat',
            'Chl',
            'socat_mask',
            'tmnth',
            'MLD_socat',
            # 'xlon_bnds', # none of the GFDL ones has these
            # 'ylat_bnds', # is that ok?
            'time_bnds',
            'MLD',
            'SSS_socat',
            'SSS',
            'SST_socat',
            'SST',
            'U10_socat',
            'U10',
            'iceFrac',
            'iceFrac_socat',
            'pATM',
            'pATM_socat',
            'pCO2_socat',
            'pCO2',
        ]
        if all([va in ds for va in required_variables]):
            dataset_dict[model][member] = ds
        else:
            print(
                (
                    f"{model}-{member} has missing variables.",
                    f"Found {list(ds.data_vars)}, missing",
                    f"{list(set(required_variables)-set(ds.data_vars))}",
                )
            )

Models:   0%|          | 0/1 [00:00<?, ?it/s]

Download Files:   0%|          | 0/25 [00:00<?, ?it/s]

Reading Files:   0%|          | 0/25 [00:00<?, ?it/s]

('GFDL-member_02 has missing variables.', "Found ['Chl_socat', 'Chl', 'socat_mask', 'tmnth', 'U10_socat', 'U10', 'iceFrac_socat', 'iceFrac'], missing", "['pCO2_socat', 'pATM_socat', 'pCO2', 'MLD', 'SSS_socat', 'MLD_socat', 'pATM', 'SSS', 'SST_socat', 'SST', 'time_bnds']")


In [56]:
# Now concat the members to a single dataset
combined_dataset_dict = {}
for model, member_dict in dataset_dict.items():
    datasets = [ds.assign_coords(member=member) for member, ds in member_dict.items()]
    if len(datasets) > 0:
        ds = xr.concat(datasets, dim='member', join='override', compat='override', coords='minimal')
        combined_dataset_dict[model] = ds

In [57]:
import gcsfs
import os
from dask.diagnostics import ProgressBar

fs = gcsfs.GCSFileSystem()
folder = f"leap-persistent/jbusecke/data/LargeEnsembleTestbed"

for model, combined_dataset in combined_dataset_dict.items():
    path = f"{folder}/{model}.zarr"
    mapper = fs.get_mapper(path)
    print(f"Saving combined dataset for {model} to bucket:{mapper.root}")
    display(combined_dataset)
    with ProgressBar():
        combined_dataset.to_zarr(
            mapper, consolidated=True, mode='w'
        )  # the last argument overwrites the existing store!

Saving combined dataset for GFDL to bucket:leap-persistent/jbusecke/data/LargeEnsembleTestbed/GFDL.zarr


<xarray.Dataset>
Dimensions:        (member: 24, time: 421, ylat: 180, xlon: 360, bnds: 2)
Coordinates:
  * time           (time) object 1982-01-16 12:00:00 ... 2017-01-16 12:00:00
  * xlon           (xlon) float64 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
  * ylat           (ylat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * member         (member) <U9 'member_01' 'member_03' ... 'member_30'
Dimensions without coordinates: bnds
Data variables: (12/21)
    Chl_socat      (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    Chl            (member, time, ylat, xlon) float32 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    socat_mask     (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    tmnth          (member, time) object dask.array<chunksize=(1, 200), meta=np.ndarray>
    MLD_socat      (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    MLD            (member, time, ylat, xlon) float32 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    ...             ...
    pATM_socat     (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    pCO2_socat     (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    pCO2           (member, time, ylat, xlon) float32 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    pCO2ATM_socat  (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    time_bnds      (member, time, bnds) object dask.array<chunksize=(1, 200, 2), meta=np.ndarray>
    pCO2ATM        (member, time, ylat, xlon) float32 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
Attributes:
    CDI:                        Climate Data Interface version 1.7.2 (http://...
    Conventions:                CF-1.4
    history:                    Wed Apr  4 13:48:28 2018: ncap2 -s Chl_socat=...
    filename:                   19500101.ocean_topaz_sfc_100.nc
    title:                      ESM2M-C1_all_historical_HC2_ens101
    grid_type:                  mosaic
    grid_tile:                  1
    NCO:                        4.6.9
    nco_openmp_thread_number:   1
    code_version:               $Name: bronx-10_performance_z1l $
    CDO:                        Climate Data Operators version 1.7.2 (http://...
    history_of_appended_files:  Wed Apr  4 13:48:25 2018: Appended file /glad...

In [60]:
import xarray as xr
import fsspec

fs = fsspec.filesystem('gcs')

In [61]:
# read a saved dataset (this works from any leap server! Try it with my test file from yours!)
path = 'gs://leap-persistent/jbusecke/data/LargeEnsembleTestbed/GFDL.zarr'
mapper = fs.get_mapper(path)
xr.open_dataset(mapper, engine='zarr', chunks={})

<xarray.Dataset>
Dimensions:        (member: 24, time: 421, ylat: 180, xlon: 360, bnds: 2)
Coordinates:
  * member         (member) <U9 'member_01' 'member_03' ... 'member_30'
  * time           (time) object 1982-01-16 12:00:00 ... 2017-01-16 12:00:00
  * xlon           (xlon) float64 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
  * ylat           (ylat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Dimensions without coordinates: bnds
Data variables: (12/21)
    Chl            (member, time, ylat, xlon) float32 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    Chl_socat      (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    MLD            (member, time, ylat, xlon) float32 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    MLD_socat      (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    SSS            (member, time, ylat, xlon) float32 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    SSS_socat      (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    ...             ...
    pCO2ATM        (member, time, ylat, xlon) float32 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    pCO2ATM_socat  (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    pCO2_socat     (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    socat_mask     (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    time_bnds      (member, time, bnds) object dask.array<chunksize=(24, 421, 2), meta=np.ndarray>
    tmnth          (member, time) datetime64[ns] dask.array<chunksize=(24, 421), meta=np.ndarray>
Attributes:
    CDI:                        Climate Data Interface version 1.7.2 (http://...
    CDO:                        Climate Data Operators version 1.7.2 (http://...
    Conventions:                CF-1.4
    NCO:                        4.6.9
    code_version:               $Name: bronx-10_performance_z1l $
    filename:                   19500101.ocean_topaz_sfc_100.nc
    grid_tile:                  1
    grid_type:                  mosaic
    history:                    Wed Apr  4 13:48:28 2018: ncap2 -s Chl_socat=...
    history_of_appended_files:  Wed Apr  4 13:48:25 2018: Appended file /glad...
    nco_openmp_thread_number:   1
    title:                      ESM2M-C1_all_historical_HC2_ens101

In [63]:
# read a saved dataset (this works from any leap server! Try it with my test file from yours!)
path = 'gs://leap-persistent/jbusecke/data/LargeEnsembleTestbed/CanESM2.zarr'
mapper = fs.get_mapper(path)
xr.open_dataset(mapper, engine='zarr', chunks={})

<xarray.Dataset>
Dimensions:        (member: 25, time: 421, ylat: 180, xlon: 360, bnds: 2)
Coordinates:
  * member         (member) <U12 'member_r1r9' 'member_r3r1' ... 'member_r5r9'
  * time           (time) object 1982-01-16 12:00:00 ... 2017-01-16 12:00:00
  * xlon           (xlon) float64 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
  * ylat           (ylat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Dimensions without coordinates: bnds
Data variables: (12/21)
    Chl            (member, time, ylat, xlon) float32 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    Chl_socat      (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    MLD            (member, time, ylat, xlon) float32 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    MLD_socat      (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    SSS            (member, time, ylat, xlon) float32 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    SSS_socat      (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    ...             ...
    pCO2_socat     (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    socat_mask     (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    time_bnds      (member, time, bnds) object dask.array<chunksize=(25, 421, 2), meta=np.ndarray>
    tmnth          (member, time) datetime64[ns] dask.array<chunksize=(25, 421), meta=np.ndarray>
    xlon_bnds      (member, xlon, bnds) float64 dask.array<chunksize=(1, 360, 2), meta=np.ndarray>
    ylat_bnds      (member, ylat, bnds) float64 dask.array<chunksize=(1, 180, 2), meta=np.ndarray>
Attributes: (12/36)
    CCCma_data_licence:         1) GRANT OF LICENCE - The Government of Canad...
    CCCma_parent_runid:         igm
    CCCma_runid:                eia-009
    CDI:                        Climate Data Interface version 1.7.2 (http://...
    CDO:                        Climate Data Operators version 1.7.2 (http://...
    Conventions:                CF-1.4
    ...                         ...
    realization:                9
    references:                 http://www.cccma.ec.gc.ca/models
    source:                     CanESM2
    table_id:                   Table Omon (22 March 2012) dc68e832d9bd926aa4...
    title:                      CanESM2 model output prepared for CanSISE his...
    tracking_id:                ae7a735f-0db8-451a-bf56-476aca62ccb6

In [64]:
# read a saved dataset (this works from any leap server! Try it with my test file from yours!)
path = 'gs://leap-persistent/jbusecke/data/LargeEnsembleTestbed/CESM.zarr'
mapper = fs.get_mapper(path)
xr.open_dataset(mapper, engine='zarr', chunks={})

<xarray.Dataset>
Dimensions:        (member: 24, time: 421, ylat: 180, xlon: 360, bnds: 2)
Coordinates:
    TLAT           float64 ...
    TLONG          float64 ...
  * member         (member) <U10 'member_002' 'member_009' ... 'member_104'
  * time           (time) object 1982-02-01 00:00:00 ... 2017-02-01 00:00:00
  * xlon           (xlon) float64 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
  * ylat           (ylat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Dimensions without coordinates: bnds
Data variables: (12/24)
    Chl            (member, time, ylat, xlon) float32 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    Chl_socat      (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    FG_CO2         (member, time, ylat, xlon) float32 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    FG_CO2_socat   (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    MLD            (member, time, ylat, xlon) float32 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    MLD_socat      (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    ...             ...
    pCO2_socat     (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    socat_mask     (member, time, ylat, xlon) float64 dask.array<chunksize=(1, 200, 180, 360), meta=np.ndarray>
    time_bnds      (member, time, bnds) object dask.array<chunksize=(24, 421, 2), meta=np.ndarray>
    tmnth          (member, time) datetime64[ns] dask.array<chunksize=(24, 421), meta=np.ndarray>
    xlon_bnds      (member, xlon, bnds) float64 dask.array<chunksize=(1, 360, 2), meta=np.ndarray>
    ylat_bnds      (member, ylat, bnds) float64 dask.array<chunksize=(1, 180, 2), meta=np.ndarray>
Attributes: (12/17)
    CDI:                        Climate Data Interface version 1.7.2 (http://...
    CDO:                        Climate Data Operators version 1.7.2 (http://...
    Conventions:                CF-1.0; http://www.cgd.ucar.edu/cms/eaton/net...
    NCO:                        4.6.9
    calendar:                   All years have exactly  365 days.
    cell_methods:               cell_methods = time: mean ==> the variable va...
    ...                         ...
    revision:                   $Id: tavg.F90 41939 2012-11-14 16:37:23Z mlev...
    source:                     CCSM POP2, the CCSM Ocean Component
    start_time:                 This dataset was created on 2013-07-26 at 16:...
    tavg_sum:                   2592000.0
    tavg_sum_qflux:             2592000.0
    title:                      b.e11.B20TRC5CNBDRD.f09_g16.002

2023-01-11 04:17:17,553 - distributed.nanny - ERROR - Worker process died unexpectedly
2023-01-11 04:17:17,553 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
2023-01-11 04:17:17,553 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2023-01-11 04:17:17,554 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.10/site-packages/distributed/nanny.py", line 981, in _run
    asyncio.run(run())
  File "/srv/conda/envs/notebook/lib/python3.10/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "/srv/conda/envs/notebook/lib/python3.10/asyncio/base_events.py", line 636, in run_until_complete
    self.run_forever()
  File "/srv/conda/envs/notebook/lib/python3.10/asyncio/base_events.p